In [18]:
import pandas as pd
import numpy as np
import gzip as gzip
import matplotlib.pyplot as plt
from datetime import datetime, date, timedelta

## Read in CoinMetrics Data

In [45]:
#Data downloaded from here https://coinmetrics.io/community-network-data/
infile='C:/Users/19084/My Backup Files/Digital Assets/btc.csv' #'F:/Proj/Digital/DigiCross/Data/Factors/coinmetrics_btc.csv.gz'
coinmetrics_df=pd.read_csv(infile)
coinmetrics_df['pre_SplyCur']=coinmetrics_df['SplyCur'].shift(1)
coinmetrics_df['BlkCntMA']=coinmetrics_df['BlkCnt'].rolling(window=5).mean()
coinmetrics_basic_df=coinmetrics_df[['time','AdrActCnt','BlkCnt','BlkCntMA','BlkSizeMeanByte','pre_SplyCur','SplyCur', \
                                     'DiffMean','IssContNtv','TxCnt','TxTfrCnt','HashRate']].copy()
coinmetrics_sample_df2=coinmetrics_basic_df.query('time>="2021-07-01" and time <="2022-06-30"')
# This avoids a lot of setting with copy warnings
coinmetrics_sample_df3=coinmetrics_sample_df2.reset_index()
coinmetrics_sample_df4=coinmetrics_sample_df3.drop(columns=['index'])
coinmetrics_sample_df=coinmetrics_sample_df4.copy()

In [46]:
coinmetrics_sample_df.head()

,time,AdrActCnt,BlkCnt,BlkCntMA,BlkSizeMeanByte,pre_SplyCur,SplyCur,DiffMean,IssContNtv,TxCnt,TxTfrCnt,HashRate
0,2021-07-01,817115.0,96.0,84.0,1.390014e+06,1.874518e+07,1.874578e+07,1.993279e+13,600.00,232056.0,708161.0,9.512298e+07
1,2021-07-02,809016.0,90.0,90.4,1.401370e+06,1.874578e+07,1.874634e+07,1.993279e+13,562.50,189904.0,627111.0,8.917780e+07
2,2021-07-03,1027108.0,127.0,98.2,1.424585e+06,1.874634e+07,1.874714e+07,1.541558e+13,793.75,234315.0,700266.0,9.732170e+07
3,2021-07-04,843475.0,115.0,103.2,1.445120e+06,1.874714e+07,1.874786e+07,1.436303e+13,718.75,169312.0,529255.0,8.210884e+07
4,2021-07-05,902032.0,132.0,112.0,1.297308e+06,1.874786e+07,1.874868e+07,1.436303e+13,825.00,223372.0,651348.0,9.424666e+07


## Set Time Interval

In [53]:
mins = 5 #set minutes here

min_diff = pd.Timedelta(mins,unit='m')
intervals = int(1440/mins)

In [54]:
## Things to estimate
minmetrics_sample_df = pd.DataFrame(index=range(0,intervals*len(coinmetrics_sample_df)))
minmetrics_sample_df['Time']=0
minmetrics_sample_df['AdrActCnt_hat']=0
minmetrics_sample_df['BlkCnt_hat']=0
minmetrics_sample_df['BlkSizeMeanByte_hat']=0
minmetrics_sample_df['DiffMean_hat']=0
minmetrics_sample_df['IssContNtv_hat']=0
minmetrics_sample_df['SplyCur_hat']=0
minmetrics_sample_df['TxCnt_hat']=0
minmetrics_sample_df['HashRate_hat']=0

In [55]:
#infile='C:/Users/19084/My Backup Files/Digital Assets/BTC/Inputs2021/202107/inputs_20210701.csv.gz'
#temp=pd.read_csv(infile)

In [56]:
#temp.head()

In [57]:
#x=temp['block_timestamp'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").replace(tzinfo=None))
#x[0]

In [58]:
proj_dir='C:/Users/19084/My Backup Files/Digital Assets/BTC/'#'F:/Proj/Digital/Network/Data/Bitcoin/'
date_list=pd.date_range("2022-01-01","2022-06-30",freq='D')
for j in range(0,len(date_list)):
    sdate=str(date_list[j])[0:10]
    print(sdate)
    sint=sdate.replace('-','')
    syear=str(sint)[0:4]
    smonth=str(sint)[4:6]
    sday=str(sint)[6:8]
    infile=proj_dir + 'Transactions' + syear + '/' + syear + smonth + '/transactions_' + sint + '.csv.gz'#'Transactions/Dataframe/' + syear + '/' + smonth + '/transactions_' + sint + '.csv.gz'
    trans_df=pd.read_csv(infile)
    infile=proj_dir + 'Blocks' + syear + '/' + syear + smonth + '/bitcoin_blocks_' + sint + '.csv.gz'#'Blocks/Dataframe/' + syear + '/' + smonth + '/blocks_' + sint + '.csv.gz'
    blocks_df=pd.read_csv(infile) #used to /blocks
    infile=proj_dir + 'Inputs' + syear + '/' + syear + smonth + '/inputs_' + sint + '.csv.gz'#'Inputs/Dataframe/' + syear + '/' + smonth +'/TSV/' + 'inputs_' + sint + '.tsv.gz'
    input_df=pd.read_csv(infile) #used to be .tsv & ,sep='\t'
    infile=proj_dir + 'Outputs' + syear + '/' + syear + smonth + '/outputs_' + sint + '.csv.gz'#'Outputs/Dataframe/' + syear + '/' + smonth +'/TSV/' + 'outputs_' + sint + '.tsv.gz'
    output_df=pd.read_csv(infile) #used to be .tsv
    pre_sup_cur = coinmetrics_sample_df['pre_SplyCur'].iloc[j]
    start_time=date_list[j]
    #minmetrics_sample_df['Time'].iloc[j]=time
    
    trans_df['times']=trans_df['block_timestamp'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))  
        #(x, "%Y-%m-%d %H:%M:%S%z").replace(tzinfo=None))   #(x, "%Y-%m-%d %H:%M:%S %Z")), one with no z
    blocks_df['times']=blocks_df['block_timestamp'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
    input_df['times']=input_df['block_timestamp'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").replace(tzinfo=None)) 
    #used to be time, and previously has no tzinfo=None clause
    output_df['times']=output_df['block_timestamp'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").replace(tzinfo=None))
    #used to be time, and previously has no tzinfo=None clause
       
    for i in range (0,intervals):
        start_time_temp = start_time+i*min_diff
        end_time = start_time+(i+1)*min_diff

        temp_trans_df=trans_df.loc[(trans_df['times'] > start_time_temp) & (trans_df['times']  <= end_time)]
        temp_blocks_df=blocks_df.loc[(blocks_df['times'] > start_time_temp) & (blocks_df['times']  <= end_time)]
        temp_input_df=input_df.loc[(input_df['times'] > start_time_temp) & (input_df['times']  <= end_time)]
        temp_output_df=output_df.loc[(output_df['times'] > start_time_temp) & (output_df['times']  <= end_time)]
        
        #Time Count
        minmetrics_sample_df['Time'].iloc[j*intervals+i]=start_time_temp
        
        # Active Address Count
        outlist=temp_output_df['recipient'].to_list()
        inlist=temp_input_df['previous_output_address'].to_list()
        address_list=inlist+outlist
        unique_address_list=np.unique(address_list)
        minmetrics_sample_df['AdrActCnt_hat'].iloc[j*intervals+i]=len(unique_address_list)
    
        # Block Count
        minmetrics_sample_df['BlkCnt_hat'].iloc[j*intervals+i]=temp_blocks_df['hash'].count()
    
        # Block Size
        minmetrics_sample_df['BlkSizeMeanByte_hat'].iloc[j*intervals+i] = temp_blocks_df['size'].sum()/len(temp_blocks_df['size'])
    
        # Mean Difficulty
        minmetrics_sample_df['DiffMean_hat'].iloc[j*intervals+i] = temp_blocks_df['difficulty'].mean()
    
        # New Issuance from Miner Rewards
        coins_per_block = 6.25 # needs to be changed for older dates
        minmetrics_sample_df['IssContNtv_hat'].iloc[j*intervals+i]=minmetrics_sample_df['BlkCnt_hat'].loc[j*intervals+i] * coins_per_block
        minmetrics_sample_df['SplyCur_hat'].iloc[j*intervals+i]=pre_sup_cur + minmetrics_sample_df['IssContNtv_hat'].iloc[j*intervals+i]
        pre_sup_cur = minmetrics_sample_df['SplyCur_hat'].iloc[j*intervals+i]
    
        # Transaction count
        minmetrics_sample_df['TxCnt_hat'].iloc[j*intervals+i]=temp_blocks_df['transaction_count'].sum()
    
        # Transaction transfer count
        # Nothing yet for BTC
    
        # HashRate
        expected_blocks = float(mins)/10 # 1,440 minutes per day / 10 minutes per block 
        block_time = 600 # 600 seconds per block
        minmetrics_sample_df['HashRate_hat'].iloc[j*intervals+i]= \
            (minmetrics_sample_df['BlkCnt_hat'].loc[j*intervals+i]) / \
            expected_blocks * (minmetrics_sample_df['DiffMean_hat'].iloc[j*intervals+i] * 2**32 / block_time / (10**12))
        

2022-01-01


C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in longlong_scalars


2022-01-02
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
2022-01-08
2022-01-09
2022-01-10
2022-01-11
2022-01-12
2022-01-13
2022-01-14
2022-01-15
2022-01-16
2022-01-17
2022-01-18
2022-01-19
2022-01-20
2022-01-21
2022-01-22
2022-01-23
2022-01-24
2022-01-25
2022-01-26
2022-01-27
2022-01-28
2022-01-29
2022-01-30
2022-01-31
2022-02-01
2022-02-02
2022-02-03
2022-02-04
2022-02-05
2022-02-06
2022-02-07
2022-02-08
2022-02-09
2022-02-10
2022-02-11
2022-02-12
2022-02-13
2022-02-14
2022-02-15
2022-02-16
2022-02-17
2022-02-18
2022-02-19
2022-02-20
2022-02-21
2022-02-22
2022-02-23
2022-02-24
2022-02-25
2022-02-26
2022-02-27
2022-02-28
2022-03-01
2022-03-02
2022-03-03
2022-03-04
2022-03-05
2022-03-06
2022-03-07
2022-03-08
2022-03-09
2022-03-10
2022-03-11
2022-03-12
2022-03-13
2022-03-14
2022-03-15
2022-03-16
2022-03-17
2022-03-18
2022-03-19
2022-03-20
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-26
2022-03-27
2022-03-28
2022-03-29
2022-03-30
2022-03-31
2022-04-01
2022-04-02

In [1]:
minmetrics_sample_df = minmetrics_sample_df[minmetrics_sample_df['Time'] != 0]

NameError: name 'minmetrics_sample_df' is not defined

In [60]:
minmetrics_sample_df.to_csv('C:/Users/19084/My Backup Files/Digital Assets/BTC/BTC_2022_5min01-06.csv')

In [ ]:
j+i


In [ ]:
minmetrics_sample_df.head()

In [38]:
minmetrics_sample_df.head()

,Time,AdrActCnt_hat,BlkCnt_hat,BlkSizeMeanByte_hat,DiffMean_hat,IssContNtv_hat,SplyCur_hat,TxCnt_hat,HashRate_hat
0,2021-04-01 00:00:00,3722,1,1299168.0,2.186556e+13,6.25,1.866998e+07,1307,3.130395e+08
1,2021-04-01 00:05:00,6429,1,1147230.0,2.186556e+13,6.25,1.867088e+07,1768,3.130395e+08
2,2021-04-01 00:10:00,0,0,NaN,NaN,0.00,1.867175e+07,0,NaN
3,2021-04-01 00:15:00,0,0,NaN,NaN,0.00,1.867257e+07,0,NaN
4,2021-04-01 00:20:00,3391,1,1152410.0,2.313744e+13,6.25,1.867349e+07,1108,3.312485e+08


In [39]:
minmetrics_sample_df.tail()

,Time,AdrActCnt_hat,BlkCnt_hat,BlkSizeMeanByte_hat,DiffMean_hat,IssContNtv_hat,SplyCur_hat,TxCnt_hat,HashRate_hat
9211,0,0,0,0.0,0.0,0.0,0.0,0,0.0
9212,0,0,0,0.0,0.0,0.0,0.0,0,0.0
9213,0,0,0,0.0,0.0,0.0,0.0,0,0.0
9214,0,0,0,0.0,0.0,0.0,0.0,0,0.0
9215,0,0,0,0.0,0.0,0.0,0.0,0,0.0


## Original Code

In [11]:
## Things to estimate
coinmetrics_sample_df['AdrActCnt_hat']=0
coinmetrics_sample_df['BlkCnt_hat']=0
coinmetrics_sample_df['BlkSizeMeanByte_hat']=0
coinmetrics_sample_df['DiffMean_hat']=0
coinmetrics_sample_df['IssContNtv_hat']=0
coinmetrics_sample_df['SplyCur_hat']=0
coinmetrics_sample_df['TxCnt_hat']=0
coinmetrics_sample_df['HashRate_hat']=0
coinmetrics_sample_df.head()

,date,AdrActCnt,BlkCnt,BlkCntMA,BlkSizeMeanByte,pre_SplyCur,SplyCur,DiffMean,IssContNtv,TxCnt,TxTfrCnt,HashRate,AdrActCnt_hat,BlkCnt_hat,BlkSizeMeanByte_hat,DiffMean_hat,IssContNtv_hat,SplyCur_hat,TxCnt_hat,HashRate_hat
0,2021-01-01,1001890,149,159.8,1.343516e+06,1.858680e+07,1.858773e+07,1.859959e+13,931.25,261318,745255,1.377640e+08,0,0,0,0,0,0,0,0


In [13]:
proj_dir='C:/Users/19084/My Backup Files/Digital Assets/BTC/'#'F:/Proj/Digital/Network/Data/Bitcoin/'
date_list=pd.date_range("2021-01-01","2021-01-01",freq='D')
for j in range(0,len(date_list)):
    sdate=str(date_list[j])[0:10]
    print(sdate)
    sint=sdate.replace('-','')
    syear=str(sint)[0:4]
    smonth=str(sint)[4:6]
    sday=str(sint)[6:8]
    infile=proj_dir + 'transactions_' + sint + '.csv.gz'#'Transactions/Dataframe/' + syear + '/' + smonth + '/transactions_' + sint + '.csv.gz'
    trans_df=pd.read_csv(infile)
    infile=proj_dir + 'blocks_' + sint + '.csv.gz'#'Blocks/Dataframe/' + syear + '/' + smonth + '/blocks_' + sint + '.csv.gz'
    blocks_df=pd.read_csv(infile)
    #blocks_df['timediff']=(blocks_df['block_timestamp']-blocks_df['block_timestamp'].shift(1))/1000

    # Active Address Count
    infile=proj_dir + 'inputs_' + sint + '.tsv.gz'#'Inputs/Dataframe/' + syear + '/' + smonth +'/TSV/' + 'inputs_' + sint + '.tsv.gz'
    input_df=pd.read_csv(infile,sep='\t')
    infile=proj_dir + 'outputs_' + sint + '.tsv.gz'#'Outputs/Dataframe/' + syear + '/' + smonth +'/TSV/' + 'outputs_' + sint + '.tsv.gz'
    output_df=pd.read_csv(infile,sep='\t')
    outlist=output_df['recipient'].to_list()
    inlist=input_df['recipient'].to_list()
    address_list=inlist+outlist
    unique_address_list=np.unique(address_list)
    coinmetrics_sample_df['AdrActCnt_hat'].iloc[j]=len(unique_address_list)
    
    # Block Count
    coinmetrics_sample_df['BlkCnt_hat'].iloc[j]=blocks_df['hash'].count()
    
    # Block Size
    coinmetrics_sample_df['BlkSizeMeanByte_hat'].iloc[j] = blocks_df['size'].sum()/len(blocks_df['size'])
    
    # Mean Difficulty
    coinmetrics_sample_df['DiffMean_hat'].iloc[j] = blocks_df['difficulty'].mean()
    
    # New Issuance from Miner Rewards
    coins_per_block = 6.25 # needs to be changed for older dates
    coinmetrics_sample_df['IssContNtv_hat'].iloc[j]=coinmetrics_sample_df['BlkCnt_hat'].loc[j] * coins_per_block
    coinmetrics_sample_df['SplyCur_hat'].iloc[j]=coinmetrics_sample_df['pre_SplyCur'].iloc[j] + coinmetrics_sample_df['IssContNtv_hat'].iloc[j]
    
    # Transaction count
    coinmetrics_sample_df['TxCnt_hat'].iloc[j]=blocks_df['transaction_count'].sum()
    
    # Transaction transfer count
    # Nothing yet for BTC
    
    # HashRate
    expected_blocks = 144 # 1,440 minutes per day / 10 minutes per block 
    block_time = 600 # 600 seconds per block
    coinmetrics_sample_df['HashRate_hat'].iloc[j]= \
        (coinmetrics_sample_df['BlkCnt_hat'].loc[j]) / expected_blocks * (coinmetrics_sample_df['DiffMean_hat'].iloc[j] * 2**32 / block_time / (10**12))

2021-01-01


In [14]:
concept_list=['AdrActCnt','BlkCnt','BlkSizeMeanByte','DiffMean','IssContNtv','SplyCur','TxCnt','HashRate'] #['AdrActCnt','TxTfrCnt']
for j in range(0,len(concept_list)):
    cname1=concept_list[j]
    cname2=concept_list[j]+ '_hat'
    cname3=concept_list[j]+ '_error'
    coinmetrics_sample_df[cname3]=coinmetrics_sample_df[cname1]-coinmetrics_sample_df[cname2]
    mean_error=coinmetrics_sample_df[cname3].mean()
    pct_error=coinmetrics_sample_df[cname3].mean()/coinmetrics_sample_df[cname1].mean()
    print(concept_list[j],"Avg. error = ",mean_error,"Pct. error = ",'{:.2f}'.format(100*pct_error))

AdrActCnt Avg. error =  -1789.0 Pct. error =  -0.18
BlkCnt Avg. error =  0.0 Pct. error =  0.00
BlkSizeMeanByte Avg. error =  -8716.72483221488 Pct. error =  -0.65
DiffMean Avg. error =  0.48828125 Pct. error =  0.00
IssContNtv Avg. error =  0.0 Pct. error =  0.00
SplyCur Avg. error =  0.0 Pct. error =  0.00
TxCnt Avg. error =  3238.0 Pct. error =  1.24
HashRate Avg. error =  -0.2958655059337616 Pct. error =  -0.00


In [15]:
coinmetrics_sample_df.head()

,date,AdrActCnt,BlkCnt,BlkCntMA,BlkSizeMeanByte,pre_SplyCur,SplyCur,DiffMean,IssContNtv,TxCnt,...,TxCnt_hat,HashRate_hat,AdrActCnt_error,BlkCnt_error,BlkSizeMeanByte_error,DiffMean_error,IssContNtv_error,SplyCur_error,TxCnt_error,HashRate_error
0,2021-01-01,1001890,149,159.8,1.343516e+06,1.858680e+07,1.858773e+07,1.859959e+13,931.25,261318,...,258080,1.377640e+08,-1789,0,-8716.724832,0.488281,0.0,0.0,3238,-0.295866
